# Genotype-Phenotype Converters
This file creates the code that allows us to convert back and forth between a bird's genotype and its phenotype

It also contains **Technical Note** sections throughout, that explain the coding and tech details of the code, and give links to external resources

## Save Genotype
First, we create a function that lets us take a dictionary of genes in the bird, and add or update the genotype for a specific gene

**Technical Note:** [A dictionary](https://docs.python.org/3/tutorial/datastructures.html#dictionaries) is a type of data in Python that contains a bunch of pieces of smaller data linked to specific words. Data can be saved under a specific name, and then retrived by using that same name, much like definitions in a paper dictionary.

In [1]:
def saveGenotypeToBird(bird, geneName, genotype):
    bird[geneName] = genotype

We can test this code by adding a new gene to a bird, and seeing that the genotype is updated.

**Technical Note:** This block of code is called a unit test, and is used to verify that the main code actually does what we want it to do.  The comments are formatted in [Gherkin](https://cucumber.io/docs/gherkin/), which is a language that is used to describe code tests in a human-readable way

**Technical Note:** If the tests pass, the code block with the tests will not produce any output, so that when we reuse these functions in another notebook, we don't change the UI.  If the tests fail, then the code block will produce errors.

In [2]:
# GIVEN an empty bird genome
bird = {}
# WHEN saveGenotypeToBird is called with a "Fake Gene" genotype of "WT/WT"
saveGenotypeToBird(bird, "Fake Gene", "WT/WT")
# THEN the bird will contain a "WT/WT" genotype for "Fake Gene"
if not bird == {'Fake Gene': 'WT/WT'}:
    raise Exception('Failed to verify genotype converter - Issue with adding gene to empty bird')

# GIVEN a bird genome with a genotype for "Fake Gene"
bird = {'Fake Gene': 'WT/WT'}
# WHEN saveGenotypeToBird is called with a "Fake Gene" genotype of "f/f"
saveGenotypeToBird(bird, "Fake Gene", "f/f")
# THEN the bird will contain a "f/f" genotype for "Fake Gene"
if not bird == {'Fake Gene': 'f/f'}:
    raise Exception('Failed to verify genotype converter - Issue with updating genotype')

## Save Phenotype
Next, we need to create a function that takes a phenotype and saves it as a genotype.  We want to make sure to reset any genotype data that conflicts with the new phenotype, and handle cases where the phenotype may require more complex genotypes. We will break this down into several smaller functions, and link them all up together to form the final function that the peafowl calculator will actually use

### Modify Simple Genes
The first step in saving the phenotype will be to update the genotype for all the autosomal genes that only have two allotypes - wild type or a single possible trait.  To do this we will check each simple gene one-at-a-time.  If the name of the simple gene matches the given phenotype, we will update the genotype for that gene to be homozygous for the trait. However, if the name of the simple gene does NOT match the phenotype, we will delete any existing genotype for that gene, so it does not conflict with the given phenotype.

(Don't worry about phenotypes that require multiple simple genes to be expressed; we will handle those in another function, and possibly deleting the genotypes that might be used in them will not prevent those genotypes from being re-updated when we handle those)

If the phenotype was found during this update, we will return a **True** value, which will allow us to 

**Technical Note:** In order to check all the simple genes, we are using a [for loop](https://docs.python.org/3/tutorial/controlflow.html#for-statements), a piece of code that runs through all items in a list, and runs the same bits of code for each item in the list

In [5]:
def updateSimpleGenesWithPhenotype(bird, phenotype, simpleGenes):
    foundPhenotype = False
    
    # Check simple genes
    for gene in simpleGenes:
        # If the current gene we are checking matches the selected
        # phenotype, set the genotype in the bird equal to homozygous
        # for the trait
        if phenotype == gene['name']:
            bird[gene['name']] = gene['notation'] + '/' + gene['notation']
            foundPhenotype = True
        # If the current gene we are checking does not match the phenotype
        # we should clear the genotype for that gene
        elif gene['name'] in bird:
            del bird[gene['name']]

    return foundPhenotype

We can test this code by creating a bird with a genotype, calling the function with a different phenotype than the one that would be displayed based on the bird's genotype, and making sure that the bird's genotype is changed to match the new phenotype

Note that in this test we check that the new genotype exists, but we also check that the old genotype was deleted

In [9]:
# GIVEN a bird genome with a genotype for "Fake Gene"
bird = {'Fake Gene': 'WT/WT'}
# AND a list of simple genes
colorGenes = [
    { 'notation': 'br',      'name': 'Bronze' },
    { 'notation': 'f',       'name': 'Fake Gene' }
]
# WHEN updateSimpleGenes is called
found = updateSimpleGenesWithPhenotype(bird, "Bronze", colorGenes)
# THEN the bird will contain a "br/br" genotype for "Bronze"
if not bird == {'Bronze': 'br/br'}:
    raise Exception('Failed to verify simple gene converter - new phenotype not added')
# AND the bird will NOT contain a genotype for "Fake Gene"
if 'Fake Gene' in bird:
    raise Exception('Failed to verify simple gene converter - new phenotype not added')

In [6]:
# This converts a phenotype to a genotype in a bird
# assuming the phenotype breeds true
def savePhenotypeToBird(bird, phenotype, sex, simpleGenes, sexLinkedTrait, 
                        sexLinkedAllotypes, multiGeneTraits, 
                        sexAndAutosomalCombos, hetSexTraits):
    foundPhenotype = updateSimpleGenesWithPhenotype(bird, phenotype, simpleGenes)

    # Reset sex-linked traits to default wild type
    if sexLinkedTrait in bird:
        del bird[sexLinkedTrait]
        
    # Check all sex-linked allotypes the same way we checked the autosomal
    # genes
    for allotype in sexLinkedAllotypes:
        if phenotype == allotype['name']:
            if sex == 'Male':
                bird[sexLinkedTrait] = allotype['notation'] + '/' + allotype['notation']
            else: 
                bird[sexLinkedTrait] = allotype['notation'] + '/w'
            foundPhenotype = True

    # At this point, all trait genes have been cleared
    # except for a matched phenotype
    # If we have found the matching phenotype already,
    # we can stop here
    if foundPhenotype:
        return

    # Save the phenotype data for genotypes that match multiple genes
    for trait in multiGeneTraits:
        if phenotype == trait['name']:
            # for each gene that makes up the multi-gene trait
            for traitGeneNotation in trait['genes']:
                # find that gene in the gene list and save it to the bird
                for gene in simpleGenes:
                    if gene['notation'] == traitGeneNotation:
                        bird[gene['name']] = gene['notation'] + '/' + gene['notation']

    # Save the phoenotype data for genes that require both autosomal and sex-linked trait
    for trait in sexAndAutosomalCombos:
        if phenotype == trait['name']:
            # Save autosomal phenotype
            for gene in simpleGenes:
                if gene['notation'] == trait['autosomalGene']:
                    bird[gene['name']] = gene['notation'] + '/' + gene['notation']

            # Save sex phenotype
            for allotype in sexLinkedAllotypes:
                if allotype['notation'] == trait['sexGene']:
                    if sex == 'Male':
                        bird[sexLinkedTrait] = allotype['notation'] + '/' + allotype['notation']
                    else: 
                        bird[sexLinkedTrait] = allotype['notation'] + '/w'
                        
    # Save phenotype for het sex traits
    for trait in hetSexTraits:
        if phenotype == trait['name'] and sex == 'Male':
            bird[sexLinkedTrait] = trait['alleles'][0] + '/' + trait['alleles'][1]
            

In [10]:
# Feature: Simple gene phenotypes work
# GIVEN a bird genome with a genotype for "Fake Gene"
bird = {'Fake Gene': 'WT/WT'}
# AND a list of simple genes
colorGenes = [
    { 'notation': 'br',      'name': 'Bronze' },
    { 'notation': 'f',       'name': 'Fake Gene' }
]
# WHEN updateSimpleGenes is called
found = savePhenotypeToBird(bird, "Bronze", 'Male', colorGenes, 'Sex-Linked Color', [], [], [], [])
# THEN the bird will contain a "br/br" genotype for "Bronze"
if not bird == {'Bronze': 'br/br'}:
    raise Exception('Failed to verify simple gene converter - new phenotype not added')
# AND the bird will NOT contain a genotype for "Fake Gene"
if 'Fake Gene' in bird:
    raise Exception('Failed to verify simple gene converter - new phenotype not added')